### Тестовое задание 
Ниже выполнено тестовое задание от некоторого оператора связи. Требовалось решить три задачи согласно условиям ниже.

### Подготовка

In [1]:
# Импорт библиотек:

import pandas as pd
!pip install pandasql
from pandasql import sqldf
import numpy as np
from numpy.random import choice
import random
from datetime import datetime, timedelta
from scipy import stats as st
import math as mth

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/tljh/user/bin/python -m pip install --upgrade pip' command.


<span style="color:blue">Необходимые для решения задач библиотеки импортированы.</span>

### Задача 1
В некотором банке запустили аб-тест на странице, где можно оформить заявку на банковскую карту. 
В варианте А из 2574 клиентов, зашедших на страницу, на баннер нажали 350, в варианте Б из 2855 нажали 375. 
Какой вариант баннера предложить продакту и почему?

In [2]:
# Для решения задачи посчитаем долю нажавших на баннер в каждом варианте,
# а затем с помощью z-теста определим, является ли различие в размере долей статистически значимым:

alpha = 0.05  # критический уровень статистической значимости

purchases = np.array([350, 375])
visits = np.array([2574, 2855])
p1 = purchases[0]/visits[0]
print('Доля клиентов в варианте А, нажавших на баннер:', round(p1, 3))
p2 = purchases[1]/visits[1]
print('Доля клиентов в варианте Б, нажавших на баннер:', round(p2, 3))
p_combined = (purchases[0] + purchases[1]) / (visits[0] + visits[1])
difference = p1 - p2 
z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/visits[0] + 1/visits[1]))
distr = st.norm(0, 1)  
p_value = (1 - distr.cdf(abs(z_value))) * 2

print('P-значение: ', p_value)

Доля клиентов в варианте А, нажавших на баннер: 0.136
Доля клиентов в варианте Б, нажавших на баннер: 0.131
P-значение:  0.6167840849677573


<span style="color:blue">Ответ: в варианте А доля клиентов, нажавших на баннер, составляет 0.136, а в варианте Б - 0.131. Таким образом, можно судить о более высокой конверсии в варианте А. Однако при проверке гипотезы о равенстве долей с помощью z-теста p-value значительно превысил заданный уровень значимости, а, значит, нулевую гипотезу отвергнуть не получилось, и между указанными выборками отсутствуют статистически значимые различия. Таким образом, различие между вариантами не является статистически значимым, и продакту можно рекомендовать любой из вариантов.</span>

### Задача 2

Есть таблица transact с транзакциями клиентов, начиная с 2023 года. В ней столбцы –
id_client - id клиента; tran_time - время транзакции; id_tran - id транзакции; sum_tran - сумма транзакции. 1) Посчитать траты каждого клиента за последние 30 дней от текущего числа. 2) Для топ-100 наиболее платежеспособных клиентов за всю историю покупок посчитать траты помесячно.

In [3]:
# Для решения задач сгенерируем датафрейм:

# зададим данные для столбцов id_client, id_tran и sum_tran, для чего сначала создадим последовательности чисел
ids_tran = np.arange(1, 1001)
ids_client = np.arange(1, 301)
sums_tran = np.arange(10, 10000)

# создадим случайный выбор 1000 чисел из вышеуказанных последовательностей
id_client    = np.random.choice(ids_client, 1000)
id_tran = np.random.choice(ids_tran, 1000)
sum_tran = np.random.choice(sums_tran, 1000)

# зададим данные для столбца tran_time, для чего укажем стартовую дату 01.01.2023
start_date = pd.to_datetime('2023-01-01')

# зададим 1000 дат, начиная со start_date, с интервалом 5 часов
tran_time = pd.date_range(start_date, periods=1000, freq='5H')

# создадим датафрейм
user_actions = pd.DataFrame({'id_client': id_client,
                             'tran_time': tran_time,
                             'id_tran': id_tran,
                             'sum_tran': sum_tran})
user_actions

,id_client,tran_time,id_tran,sum_tran
0,251,2023-01-01 00:00:00,284,4630
1,131,2023-01-01 05:00:00,928,580
2,35,2023-01-01 10:00:00,241,9241
3,58,2023-01-01 15:00:00,689,4564
4,185,2023-01-01 20:00:00,870,5568
...,...,...,...,...
995,252,2023-07-27 07:00:00,882,2723
996,109,2023-07-27 12:00:00,144,7161
997,211,2023-07-27 17:00:00,563,3541
998,82,2023-07-27 22:00:00,185,9693


<span style="color:blue">Датафрейм готов, можно приступать к решению задач.</span>

In [4]:
# Задача 1. Посчитать траты каждого клиента за последние 30 дней от текущего числа:

q1 = """   SELECT id_client,
                  SUM(sum_tran) full_sum_tran
           FROM user_actions
           WHERE STRFTIME('%Y-%m-%d', tran_time)  >= (SELECT DATE('NOW', '-30 day'))
           GROUP BY id_client
           ORDER BY full_sum_tran DESC;"""


In [5]:
sqldf(q1)

,id_client,full_sum_tran
0,223,36735
1,163,35012
2,27,33633
3,242,31942
4,24,30947
...,...,...
230,279,823
231,77,439
232,285,339
233,134,319


In [6]:
# Задача 2. Для топ-100 наиболее платежеспособных клиентов за всю историю покупок посчитать траты помесячно:

q2 = """   SELECT id_client,
                  strftime('%m', tran_time) month, 
                  SUM(sum_tran) sum_tran_per_month
           FROM user_actions
           WHERE id_client IN (SELECT id_client
                               FROM user_actions
                               GROUP BY id_client
                               ORDER BY SUM(sum_tran) DESC
                               LIMIT 100)
           GROUP BY id_client, strftime('%m', tran_time);"""

In [7]:
sqldf(q2)

,id_client,month,sum_tran_per_month
0,3,03,6352
1,3,04,8086
2,3,05,6894
3,5,01,3545
4,5,03,2291
...,...,...,...
367,300,03,7441
368,300,04,3258
369,300,05,6118
370,300,06,2923


<span style="color:blue">Задачи решены.</span>

### Задача 3
Пришел продукт и говорит: "хочу посчитать retention клиентов из задачи 2". 
Опиши, пожалуйста, твои действия - например, вопросы которые ты задашь. 
Предположи ответы на свои вопросы, и исходя из этих предположений, напиши код в Python или Sql, 
который поможет достать нужные данные.

In [8]:
# Для расчета retention rate у продакта необходимо узнать момент и горизонт анализа.
# Возьмем 31.01.2023 за момент анализа и 10 дней за горизонт анализа.

# преобразуем данные о времени для дальнейших расчётов
user_actions['tran_time_date'] = pd.to_datetime(user_actions['tran_time']).dt.date

# отсортируем данные по id_client и tran_time
user_actions = user_actions.sort_values(by = ['id_client', 'tran_time']).reset_index(drop=True)

# сформируем столбец с датой первой операции для каждого пользователя
user_actions['min_tran_time'] = user_actions.groupby('id_client')['tran_time'].transform('first')

# зададим момент и горизонт анализа данных
observation_date = datetime(2023, 1, 31).date()
horizon_days = 10

# создадим опцию «игнорировать горизонт»
ignore_horizon = False

# рассчитаем максимально возможную дату привлечения
last_suitable_acquisition_date = observation_date
if not ignore_horizon:
    last_suitable_acquisition_date = observation_date - timedelta(days=horizon_days - 1)

# исключим пользователей, пришедших после максимальной даты привлечения
user_actions = user_actions.query('tran_time_date <= @last_suitable_acquisition_date')

# вычислим лайфтайм каждой сессии в днях
user_actions['lifetime'] = (user_actions['tran_time'] - user_actions['min_tran_time']).dt.days

# построим таблицу удержания
user_actions_grouped = user_actions.pivot_table(
    index=['tran_time_date'], columns='lifetime', values='id_client', aggfunc='nunique'
)

# вычислим размеры когорт
cohort_sizes = (
    user_actions.groupby('tran_time_date')
    .agg({'id_client': 'nunique'})
    .rename(columns={'id_client': 'cohort_size'})
)

# объединим размеры когорт и таблицу удержания
user_actions_grouped = cohort_sizes.merge(
    user_actions_grouped, on='tran_time_date', how='left'
).fillna(0)

# поделим данные таблицы удержания на размеры когорт
user_actions_grouped = user_actions_grouped.div(
    user_actions_grouped['cohort_size'], axis=0
).drop(columns=['cohort_size'])
user_actions_grouped

,0,1,2,3,4,5,7,8,13,15,16,19
tran_time_date,,,,,,,,,,,,
2023-01-01,1.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2023-01-02,1.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2023-01-03,1.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2023-01-04,0.80,0.2,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2023-01-05,1.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2023-01-06,0.60,0.0,0.4,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2023-01-07,0.60,0.0,0.0,0.4,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2023-01-08,0.80,0.0,0.0,0.2,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0
2023-01-09,0.60,0.0,0.0,0.2,0.2,0.00,0.00,0.0,0.00,0.0,0.0,0.0


<span style="color:blue">Retention rate посчитан, все задачи решены.</span>